**This notebook contains a function that fetches a SMILE string starting from a KEGG compound page. It also contains the associated unittests.**

USE KEGG_DF_TO_SMILES and SID_TO_SMILES

All of the unit tests are passing as of 3/5/19 at 7:30 pm. 

This notebook contains the basis of a function(s) to take a PubChem ID number and fetch the associated SMILES string from PubChem.

It also contains code pieces to pull an SID from a KEGG webpage.

In [1]:
import numpy as np
import pandas as pd
import pubchempy as pc
import requests
import re
from time import sleep

from bs4 import BeautifulSoup


There are multiple identifier types for each chemical in PubChem. The two we are interacting with here are **SID** (substance ID) and **CID** (chemical ID). CID can be used to acces SMILES directly with PubChemPy. **KEGG does not have CID**, only SID. SID can be turned into CID from which SMILES can be found. 

### Get SMILES from CID and SID

In [3]:
# get SMILES directly from CID
for compound in pc.get_compounds('243'):
    print(compound.isomeric_smiles)

C1=CC=C(C=C1)C(=O)O


In [31]:
# get SMILES from SID through mapping to CID
substance = pc.Substance.from_sid('3371')
cid = substance.standardized_cid
if cid == None:
    pass
    none_tally += 1
else:
    compound = pc.get_compounds(cid)[0]

print(none_tally)
#print(compound.isomeric_smiles, compound)

5


In [35]:
def sid_to_smiles(df):
    """Takes an SID and prints the associated SMILES string."""
    sid = df['SID']
    substance = pc.Substance.from_sid(sid)
    cid = substance.standardized_cid
    if cid == None:
        pass
        none_tally += 1
    else:
        compound = pc.get_compounds(cid)[0]

    return compound.isomeric_smiles

In [ ]:
# for compound in pc.get_compounds('glucose', 'name'):
#    print(compound.cid, compound.isomeric_smiles, compound.smiles)

In [2]:
#%%writefile make_smiles_utils.py

import pubchempy as pc
# I could easily make this handle SIDs, too, but then the user would have to specify whether it is an SID or a CID.
def user_input_to_smiles(input_cid):
    """Takes a PubChem CID input and outputs the associated SMILES."""
    assert type(input_cid) is int, 'Expected an integer ID input'
    for compound in pc.get_compounds(input_cid):
        print(compound.isomeric_smiles)

Overwriting make_smiles_utils.py


In [72]:
user_input_to_smiles(243)

C1=CC=C(C=C1)C(=O)O


In [33]:
#%%writefile test_make_smiles_utils.py

import pubchempy as pc
import make_smiles_utils

def test_user_input_to_smiles():
    # check that the input is an integer
    # check that the output is a string
    
    # CID for thiophene
    inp_cid = 8030 
    smiles = make_smiles_utils.user_input_to_smiles(inp_cid)
    
    assert len(str(smiles)) == 4, 'This is not the correct SMILES length for thiophene'
    assert str(smiles) == 'C1=CSC=C1', 'This is not the correct SMILES for thiophene'
    
    return #len(str(smiles))

In [35]:
test_user_input_to_smiles()

C1=CSC=C1


__________________

### Manipulate DF containing SID into SMILES

In [107]:
compound_df = pd.read_csv('../datasets/promiscuous_cleaned_KEGGtoPubChem.csv')

In [128]:
devo_df.SID.astype(int)

ValueError: invalid literal for int() with base 10: '3480.0'

In [125]:
for i in range(len(devo_df)):
    try:
        devo_df.SID.astype(int)
    except:
        devo_df.iat[i, 1] = 0
        
devo_df.head()

,Compound Name,SID
0,NAD,0
1,glucose,0
2,benzoic acid,0
3,unknown,3371
4,methanol,3432


In [115]:
for i in range(len(devo_df)):
    try:
        a = devo_df.iloc[i,1]
        value = int(a)
        devo_df.iat[i, 1] = value
    except:
        devo_df.iat[i, 1] = 0
devo_df.head()

,Compound Name,SID
0,NAD,3305
1,glucose,3333
2,benzoic acid,0
3,unknown,3371
4,methanol,3432


In [110]:
value = int(compound_df.iloc[0,4])
value

3371

In [3]:
sids = ['3305', '3333', '3480', '3371']
compounds = ['NAD', 'glucose', 'benzoic acid', 'unknown']
tuple_list = list(zip(compounds, sids))
tuple_list
devo_df = pd.DataFrame(tuple_list, columns=['Compound Name','SID'])
devo_df

,Compound Name,SID
0,NAD,3305
1,glucose,3333
2,benzoic acid,3480
3,unknown,3371


In [6]:
def sid_to_smiles(sid):
    """Takes an SID and prints the associated SMILES string."""

    substance = pc.Substance.from_sid(sid)
    cid = substance.standardized_cid
    compound = pc.get_compounds(cid)[0]
        
    
    return compound.isomeric_smiles, cid


def kegg_df_to_smiles(kegg_df):
    """Takes a pandas dataframe that includes a column of SIDs, gets the isomeric SMILES for each SID, stores them as a list, then adds a SMILES column."""

    res = [] 
    cid_list = []
    for i in range(len(kegg_df)):
        sid = kegg_df.iloc[i, 1] #CHANGE THIS 1 TO THE PROPER COLUMN NUMBER FOR SID
        try:
            smile_result = sid_to_smiles(sid)[0]
            res.append(smile_result)
            cid_result = sid_to_smiles(sid)[1]
            cid_list.append(cid_result)
        except:
            res.append('none')
            cid_list.append('none')
            pass
    kegg_df.insert(2, column='CID', value=cid_list)    
    kegg_df.insert(3, column='SMILES', value=res) #Change this 2 to the number where the smiles column should be
    
    return kegg_df

In [7]:
kegg_df_to_smiles(devo_df)

,Compound Name,SID,CID,SMILES
0,NAD,3305,5893,C1=CC(=C[N+](=C1)[C@H]2[C@@H]([C@@H]([C@H](O2)...
1,glucose,3333,5793,C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)O)O)O)O
2,benzoic acid,3480,243,C1=CC=C(C=C1)C(=O)O
3,unknown,3371,none,none


In [79]:
exp_df = compound_df.tail(100)
exp_df

,Unnamed: 0,entry,KEGG,formula,PubChem
2044,2094,5.4.2.11,C01159,C3H8O10P2,4387.0
2045,2095,5.4.2.11,C00631,C3H7O7P,3904.0
2046,2098,5.4.3.3,C01186,C6H14N2O2,4412.0
2047,2097,5.4.3.3,C00739,C6H14N2O2,4002.0
2048,2099,5.4.3.3,C05161,C6H14N2O2,7577.0
2049,2100,5.4.3.3,C01142,C6H14N2O2,4373.0
2050,1393,5.5.1.24,C14156,C27H40O2,7846968.0
2051,1391,5.5.1.24,C14151,C27H46O2,7846963.0
2052,1389,5.5.1.24,C14155,C28H42O2,7846967.0
2053,1396,5.5.1.24,C02483,C28H48O2,5496.0


In [81]:
kegg_df_to_smiles(exp_df)

ValueError: cannot convert float NaN to integer

In [129]:
#%%writefile kegg_data.py

import pandas as pd
import pubchempy as pc

def sid_to_smiles(sid):
    """Takes an SID and prints the associated SMILES string."""
    
    substance = pc.Substance.from_sid(sid)
    cid = substance.standardized_cid
    compound = pc.get_compounds(cid)[0]
    return compound.isomeric_smiles


def kegg_df_to_smiles(kegg_df):
    """Takes a pandas dataframe that includes a column of SIDs, gets the isomeric SMILES for each SID, stores them as a list, then adds a SMILES column."""

    res = [] 
    
    for i in range(len(kegg_df)):
        sid = kegg_df.iloc[i, 1] #CHANGE THIS 1 TO THE PROPER COLUMN NUMBER FOR SID 
        result = sid_to_smiles(sid)
        res.append(result)
        
    
    kegg_df.insert(2, column='SMILES', value=res) #Change this 2 to the number where the smiles column should be
    
    return kegg_df

Overwriting kegg_data.py


In [112]:
kegg_df_to_smiles(devo_df)

,Compound Name,SID,SMILES
0,NAD,3305,C1=CC(=C[N+](=C1)[C@H]2[C@@H]([C@@H]([C@H](O2)...
1,glucose,3333,C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)O)O)O)O
2,benzoic acid,3480,C1=CC=C(C=C1)C(=O)O
3,methanol,3432,CO


In [128]:
#%%writefile test_kegg_data.py

import pandas as pd
import pubchempy as pc

import kegg_data

def test_sid_to_smiles():
    
    sids = ['3489', '3990']
    expected = ['C(CO)N', 'C1CSSC1CCCCC(=O)O']
    actual = []
    
    for sid in sids:
        result_smile = kegg_data.sid_to_smiles(sid)
        
        assert len(result_smile) >= 1, 'SMILES string is very short. Check SMILES.'
        isinstance(result_smile, str), 'SMILES not returned as string.'
        
        actual.append(result_smile)
    
    assert expected == actual, 'Actual SMILES are not the expected SMILES.'
    
    return
   
    
    
def test_kegg_df_to_smiles():
    
    test_frame = pd.DataFrame([['ethanolamine', '3489'], ['pyruvate', '3324']], columns=['Compound Name', 'SID'])
    
    expected_frame = pd.DataFrame([['ethanolamine', '3489', 'C(CO)N'], ['pyruvate', '3324', 'CC(=O)C(=O)O']], columns=['Compound Name', 'SID', 'SMILES'])
    
    result_frame = kegg_data.kegg_df_to_smiles(test_frame)
    
    assert result_frame.equals(expected_frame), 'Did not generate expected df.'
    
    return


Overwriting test_kegg_data.py


---
### Get SID from KEGG url 

### Probably don't need this stuff, keeping it just in case

This currently works from the compound page. I have not seen if it can be pulled from the reaction page.


In [118]:
#%%writefile kegg_utils.py

import pubchempy as pc
import re
import requests

from bs4 import BeautifulSoup


def sid_to_smiles(sid):
    substance = pc.Substance.from_sid(sid)
    cid = substance.standardized_cid
    compound = pc.get_compounds(cid)[0]
    return compound.isomeric_smiles


def kegg_to_sid(url):
    # access the url
    response = requests.get(url)

    # turn the webpage into html
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the link that contains 'pubchem'
    sid = soup.find('a', href=re.compile(r'https://pubchem\.ncbi'))

    sid_string = sid.string

    return sid_string


def kegg_to_smiles(url):
    """Uses the KEGG compound page url to find the compound's PubChem SID, then to find the SMILES for that compound using the SID."""

    # access the url
    response = requests.get(url)

    # turn the webpage into html
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the link that contains 'pubchem'
    sid = soup.find('a', href=re.compile(r'https://pubchem\.ncbi'))

    substance = pc.Substance.from_sid(sid.string)
    cid = substance.standardized_cid
    compound = pc.get_compounds(cid)[0]

    print(compound.isomeric_smiles)

Overwriting kegg_utils.py


In [ ]:
#%%writefile kegg_utils.py

import pubchempy as pc
import re
import requests

from bs4 import BeautifulSoup


def kegg_to_sid(url):
    # access the url
    response = requests.get(url)

    # turn the webpage into html
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the link that contains 'pubchem'
    sid = soup.find('a', href=re.compile(r'https://pubchem\.ncbi'))

    sid_string = sid.string

    return sid_string

In [7]:
# url of the desired KEGG compound page
url = 'https://www.genome.jp/dbget-bin/www_bget?cpd:C00180'
# access the url
response = requests.get(url)

# turn the webpage into html
soup = BeautifulSoup(response.content, 'html.parser')

# find the link that contains 'pubchem'
sid = soup.find('a', href=re.compile('https://pubchem\.ncbi'))

# print the string that is displayed as the link
# (this is the SID, which works with pubchempy to get the SMILES)
print(sid.string)

3480


In [119]:
#%%writefile test_kegg_utils.py
import unittest

import kegg_utils


def test_kegg_to_sid():
    url_list = [
        'https://www.genome.jp/dbget-bin/www_bget?cpd:C00180',
        'https://www.genome.jp/dbget-bin/www_bget?cpd:C00587',
        'https://www.genome.jp/dbget-bin/www_bget?cpd:C00002']
    for url in url_list:
        sid_str = kegg_utils.kegg_to_sid(url)
        sid_str.isdigit(), 'SID contains characters other than numbers'
    return


def test_kegg_to_smiles():

    url = 'https://www.genome.jp/dbget-bin/www_bget?cpd:C00587'
    smiles = kegg_utils.kegg_to_sid(url)

    assert len(smiles) >= 1, 'SMILES string is very short. Check SMILES.'
    isinstance(smiles, str), 'SMILES not returned as string.'
    
    return

import pandas as pd
import pubchempy as pc

import kegg_utils

def test_kegg_df_to_smiles():
    
    test_frame = pd.DataFrame([['ethanolamine', '3489'], ['pyruvate', '3324']], columns=['Compound Name', 'SID'])
    
    expected_frame = pd.DataFrame([['ethanolamine', '3489', 'C(CO)N'], ['pyruvate', '3324', 'CC(=O)C(=O)O']], columns=['Compound Name', 'SID', 'SMILES'])
    
    result_frame = kegg_utils.kegg_df_to_smiles(test_frame)
    
    assert result_frame.equals(expected_frame), 'Did not generate expected df.'
    
    return

Overwriting test_kegg_utils.py


biopython kegg api to pull SID

store smiles into dataframe to join later 
